In Python, we know that attribute refers to data attribute and methods.

We could have two types of attribute:
1. class attribute<br>
   Any attribute that is defined inside the class body and **not** inside the body of `__init__` method
2. instance attribute<br>
   Attributes that are defined inside the body of `__init__` method
   
```python
class MyClass:
    class_attribute = None
    
    def __init__(self):
        instance_attribute = None
```

This means even before we instantiate a class, we need to define its class's attributes and instance's attribute. What if you don't know about that in advance? This is where we need `dynamic attribute`. A typical use case for `dynamic attribute` is to provide a convenient way to access data of a dictionary via normal attribute access.

Let's say that you have a `dict` like below:

```python
    my_class = {
        'primary_teacher': "Professor Tom",
        'number_of_student': 40,
        'class_name': 'Python 101'
    }
    
    print(my_class['primary_teacher'])
    print(my_class['class_name']
```

It will be more convenient to access the element like this:
```python
    print(my_class.primary_teacher)
    print(my_class.class_name)
```
It feels more object oriented this way, right?

You could always create a class with these attributes. However, if you're only trying to explore the data, or you have to deal with arbitrarily data on a regular basis, create a class for each data structure will be boresome and tedious.

Below is a class that could help you create such obj

In [6]:
class DataReader:
    
    def __init__(self, mapping):
        self.__data = mapping
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return self.__data.name
        else:
            try:
                return self.__data[name]
            except KeyError:
                msg = "Cannot find attr with name {}"
                raise AttributeError(msg.format(name))
                
my_class = {
        'primary_teacher': "Professor Tom",
        'number_of_student': 40,
        'class_name': 'Python 101'
    }

my_data = DataReader(my_class)
print(my_data.primary_teacher)

print(my_data.unknown_attribute)

Professor Tom


AttributeError: Cannot find attr with name unknown_attribute

The simple class above works by leveraging the method `__getattr__` which will always be called if AttributeError was raised during the attempt to access an attribute.

Obviously, the instance `my_data` of `DataReader` doesn't have any methods or attributes whatsoever. So when we're try to access an attribute, without `__getattr__`, the exception AttributeError will be raised.

In [8]:
class DataReader:
    
    def __init__(self, mapping):
        self.__data = mapping

my_class = {
    'primary_teacher': "Professor Tom",
    'number_of_student': 40,
    'class_name': 'Python 101'
}
my_data = DataReader(my_class)
print(my_data.primary_teacher)

AttributeError: 'DataReader' object has no attribute 'primary_teacher'

But we do have `__getattr__` implemented, don't we? When AttributeError is raised, `__getattr__` is called and eventually look for the unknown attribute in our only attribute `__data` which is holding a mapping. By the end, if the attribute is indeed a key in that mapping, we successfully retrieve a value. So we could basically create an `DataReader` instance from any mapping and support data access via dot notation.<br>
Next, we will enhance our DataReader to deal with nested dictionary

In [14]:
from collections import MutableMapping

class DataReaderV2:
    
    def __init__(self, mapping):
        self.__data = mapping
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return self.__data.name
        else:
            try:
                if isinstance(self.__data[name], MutableMapping):
                    return DataReaderV2(self.__data[name])
                elif isinstance(self.__data[name], list):
                    return [DataReaderV2(data) for data in self.__data[name]]
                else:
                    return self.__data[name]
            except KeyError:
                msg = "Cannot find attr with name {}"
                raise AttributeError(msg.format(name))
                
my_class = {
    'primary_teacher': {
        'name': "Professor Tom",
        'age': 35,
    },
    'number_of_student': 40,
    'class_name': 'Python 101',
    'students': [
        {
            'name': 'An',
            'age': 18,
        },
        {
            'name': 'Vien',
            'age': 18,
        },
    ]
}

my_data = DataReaderV2(my_class)
print(my_data.primary_teacher.name)
print(my_data.students[0].age)
print(dir(my_data))

Professor Tom
18
['_DataReaderV2__data', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


In [13]:
class DataReaderV3:
    
    def __init__(self, mapping):
        self.__dict__.update(mapping)
        
my_class = {
    'primary_teacher': "Professor Tom",
    'number_of_student': 40,
    'class_name': 'Python 101'
}

my_data = DataReaderV3(my_class)
print(my_data.primary_teacher)
print(dir(my_data))

Professor Tom
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'class_name', 'number_of_student', 'primary_teacher']


This version is much shorter but won't support nested.

The line
```python
self.__dict__.update(mapping)
```

is actually a Python idiom, used to quickly create many `dynamic attribute` at once.<br>
Because all instance attribute are stored in `__dict__`, so updating `__dict__` is equivalent to adding attribute to the objects.

Next concept is property and property factory

`property` is an implementation of Data Discriptor protocol

In [4]:
class MyClass:
    
    def __init__(self, initial_value):
        self.__msg = "Hello " + str(initial_value)
    @property
    def greeting(self):
        return self.__msg
    
my_object = MyClass("An")
print(my_object.greeting)

my_object.greeting = "Hello Vien"

Hello An


AttributeError: can't set attribute

In [3]:
class MyClassV2:
    
    def __init__(self, initial_value):
        self.__msg = "Hello " + str(initial_value)
        
    @property
    def greeting(self):
        return self.__msg
    
    @greeting.setter
    def greeting(self, new_value):
        self.__msg = "Hello " + str(new_value)
    
my_object = MyClassV2("An")
print(type(my_object.greeting))
print(my_object.greeting)

my_object.greeting = "Vien"
print(my_object.greeting)

<class 'str'>
Hello An
Hello Vien


In [22]:
class MyProperty:
    
    def __init__(self, getter=None, setter=None, deleter=None, description=None):
        self._getter = getter
        self._setter = setter
        self._deleter = deleter
        self._description = description
    
    def __get__(self, obj, objtype=None):
        print('__get__ being called')
        return self._getter(obj)
    
    def __set__(self, obj, value):
        self._setter(obj, value)
    
    def setter(self, setter):
        return type(self)(getter=self._getter, setter=setter, deleter=self._deleter, description=self._description)

def my_property(func):
    obj = MyProperty(getter=func)
    print(type(obj))
    
    return obj


class MyClassV3:
    
    normal_property = MyProperty()
    
    def __init__(self, initial_value):
        self.__msg = "Hello " + str(initial_value)
        
    @my_property
    def greeting(self):
        return self.__msg
    
    @greeting.setter
    def greeting(self, new_value):
        self.__msg = "Hello " + str(new_value)
        
    print(greeting)
        
my_object = MyClassV3("An")
print(type(my_object.greeting))
print(my_object.greeting)

my_object.greeting = "Vien"
print(my_object.greeting)

<class '__main__.MyProperty'>
__get__ being called
<class 'str'>
__get__ being called
Hello An
__get__ being called
Hello Vien


# property factory

It's a factory that create property obj. Just that simple

In [26]:
class People():
    
    def __init__(self, salary=1000):
        self._salary = salary
    
    @property
    def salary(self):
        return self._salary
    
    @salary.setter
    def salary(self, value):
        self._salary = value
        
people = People()

print(people.salary)
people.salary = -100
print(people.salary)

1000
-100


It wouldn't very sensible if someone has negative salary. We'd like to have some validation here.

In [ ]:
class PeopleV2():
    
    def __init__(self, salary=1000):
        self._salary = salary
    
    @property
    def salary(self):
        return self._salary
    
    @salary.setter
    def salary(self, value):
        if value > 0:
            self._salary = value
        else:
            raise ValueError("Salary must greater than 0")
        
people = PeopleV2()

print(people.salary)
people.salary = -100
print(people.salary)

Good, but apart from salary, there must be other attributes that don't accept negative value, such as age, height, etc.. So we just need to go ahead and do the same validation in `property.setter`, easy enough right?<br>
Yes and No, repeating your self doesn't only make your code long, but also reduce reusability of your code. Everytime you need to reuse this validation logic, you need to copy it over. In case your logic change, just think of how many place you need to update the logic.

So, we need to think about away to reuse our logic. Since we're creating property() obj, it's natural to think of some sort of factory that mass produce these obj for us.

In [12]:
def my_factory(name):
    def setter(obj, value):
        if value > 0:
            obj.__dict__[name] = value
        else:
            raise ValueError(f"{name} must greater than 0")
            
    def getter(obj, objtype=None):
        return obj.__dict__[name]
    
    return property(getter, setter)


class PeopleV3:
    
    salary = my_factory('salary')
    age = my_factory('age')
    
    def __init__(self, salary=100, age=1):
        self.salary = 100
        self.age = age
    
people = PeopleV3()
print(people.salary)
people.salary = -1

100


ValueError: salary must greater than 0

In [42]:
class DataDescriptor:
    __call_count = 0
    def __init__(self, storage_name):
        self.storage_name = storage_name  # the constructor accept a variable name and store it for later use
        
    @classmethod
    def call_count(cls):
        cls.__call_count += 1
        
        return cls.__call_count
        
    def __get__(self, instance, owner):
        print(f'#{self.call_count()} __get__({self}, {instance}, {owner})')
        # retrieve the storage name from instance __dict__
        return instance.__dict__[self.storage_name]
    
    def __set__(self, instance, value):
        print(f'#{self.call_count()} __set__({self}, {instance}, {value!r})')
        # store the value into the instance's __dict__
        # under the the name of the variable which is stored earlier
        instance.__dict__[self.storage_name] = value
        
class ManagedClass:
    managed_attribute = DataDescriptor('managed_attribute')  # have to pass the variable name to the constructor
    another_attribute = DataDescriptor('another_attribute')  # have to pass the variable name to the constructor
    
    def __init__(self, value):
        self.managed_attribute = value
        
        
managed_instance = ManagedClass('My name') # 1
print(managed_instance.managed_attribute) # 2
managed_instance. another_attribute = "value for another_attribute" # 3
print(managed_instance.another_attribute) # 4

#1 __set__(<__main__.DataDescriptor object at 0x00000294D7D40D90>, <__main__.ManagedClass object at 0x00000294D7D40D30>, 'My name')
#2 __get__(<__main__.DataDescriptor object at 0x00000294D7D40D90>, <__main__.ManagedClass object at 0x00000294D7D40D30>, <class '__main__.ManagedClass'>)
My name
#3 __set__(<__main__.DataDescriptor object at 0x00000294D7D40490>, <__main__.ManagedClass object at 0x00000294D7D40D30>, 'value for another_attribute')
#4 __get__(<__main__.DataDescriptor object at 0x00000294D7D40490>, <__main__.ManagedClass object at 0x00000294D7D40D30>, <class '__main__.ManagedClass'>)
value for another_attribute


In [29]:
class DataDescriptorV2:
    
    def __init__(self, storage_name):
        self.storage_name = storage_name
        
    def __get__(self, instance, owner):
        print(f'__get__({self}, {instance}, {owner})')
        return 42
    
    def __set__(self, instance, value):
        print(f'__set__({self}, {instance}, {value})')
        instance.__dict__[self.storage_name] = value
        
class ManagedClassV2:
    managed_attribute = DataDescriptorV2('managed_attribute')
    another_attribute = DataDescriptorV2('another_attribute')
    
    def __init__(self, value):
        self.managed_attribute = value
        
        
managed_instance = ManagedClassV2('My name')
print(type(managed_instance.managed_attribute))
print(managed_instance.managed_attribute)
print(dir(managed_instance.managed_attribute))
print(managed_instance.another_attribute)
print(ManagedClass.another_attribute)

__set__(<__main__.DataDescriptorV2 object at 0x00000294D7D5B0D0>, <__main__.ManagedClassV2 object at 0x00000294D7D5B430>, My name)
__get__(<__main__.DataDescriptorV2 object at 0x00000294D7D5B0D0>, <__main__.ManagedClassV2 object at 0x00000294D7D5B430>, <class '__main__.ManagedClassV2'>)
<class 'int'>
__get__(<__main__.DataDescriptorV2 object at 0x00000294D7D5B0D0>, <__main__.ManagedClassV2 object at 0x00000294D7D5B430>, <class '__main__.ManagedClassV2'>)
42
__get__(<__main__.DataDescriptorV2 object at 0x00000294D7D5B0D0>, <__main__.ManagedClassV2 object at 0x00000294D7D5B430>, <class '__main__.ManagedClassV2'>)
['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__mul_

In [33]:
class DataDescriptorV3:
    __count = 0
    
    def __init__(self):
        self.storage_name = "attribute" + str(DataDescriptorV3.__count)
        DataDescriptorV3.__count += 1
        

    
    def __set__(self, instance, value):
        print(f'__set__({self}, {instance}, {value})')
        instance.__dict__[self.storage_name] = value
        
class ManagedClassV3:
    managed_attribute = DataDescriptorV3()
    another_attribute = DataDescriptorV3()
    
    def __init__(self, value):
        self.managed_attribute = value
        
        
managed_instance = ManagedClassV3('My name')
print(dir(managed_instance))
print(managed_instance.managed_attribute)
print(type(managed_instance))
print(managed_instance.__dict__['managed_attribute'])
print(dir(managed_instance.managed_attribute))
print(managed_instance.another_attribute)
print(ManagedClass.another_attribute)

__set__(<__main__.DataDescriptorV3 object at 0x00000294D78AAC10>, <__main__.ManagedClassV3 object at 0x00000294D78AADF0>, My name)
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'another_attribute', 'attribute0', 'managed_attribute']
<class '__main__.ManagedClassV3'>


In [32]:
class Cache:
    
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, owner):
        print('__get__')
        result = "this is result"
        instance.__dict__[self.name] = result
        
        return result
    
class MyClass:
    
    my_attr = Cache("my_attr")
    
    def show(self):
        print(self.my_attr)
        
my_obj = MyClass()
my_obj.show()
my_obj.show()
my_obj.show()

__get__
this is result
this is result
this is result


Notice that the `__get__()` method is called only one time. The result was stored in the instance attribute with the same name. Consequent access to this attribute doesn't invoke the descriptor anymore, but retrieves directly from the instance attribute.

Pseudo code of function in Python:

```python

class BoundMethod:
    
    def __init__(self, func, obj):
        self.func = func  # store the func
        self.obj = obj  # store the obj (which is the instance of a class)
        
    def __call__(self, *args, **kwargs):
        return self.func(self.obj, *args, **kwargs)  # Execute the bound method when we call: obj.method(*args, **kwargs)

class function:
    
    def __init__(self):
        pass
    
    def __get__(self, instance, owner):
        # instance is none means that this is accessed via a class, something like Klass.function
        if instance is None: 
            return self  # return the function itself.
        else:
            return BoundMethod(self, instance)  # create a bound method object that wrap itself, and the instance
        
    # There will be no __set__ because function is a non-overriding descriptor.
    # You could always monkey-patching a function by writing: instance.method = something
        
    def __call__(self, *args, **kwargs):
        return self(*args, **kwargs)
```

The fact that function is a non-overriding descriptor is mindblowing to me the first time I learn it. There's something really clever and sophisticated about this implementation:<br>
With a simple descriptor, you could turn an original class (the `function` class in this case) into a different object depends on the usecases:
1. If invoked by a class, it's a function
2. If access via an instance, it's a bound method of that instance.

Can we apply this pattern to other usecases?
Let's analyze it first:
1. You know about `instance` and `owner` so any conditions you could form using these two factors could be used to branch the flow.
2. You will want to return something that is similar to the current obj.

```python
    class classmethod(func):
        
        def __init__(self, func):
            self._func = func
            
        def __get__(self, instance, owner):
            if instance is None:
                return BoundMethod(owner, func)
            else:
                return BoundMethod(type(instance), func)
            
    class staticmethod(func):
        
        def __init__(self, func):
            self._func = func
            
        def __get__(self, instance, owner):
            # Simple return the function itself. Upon invocation, no special argument was passed into it.
            return func
```

In [72]:
class Klass():

    print(classmethod)
    @classmethod
    def show(kls):
        print('klass')
        
    print(staticmethod)
    @staticmethod
    def show_static():
        print('static')
        
    def normal_method(self):
        pass
        
    print(show_static)
    print(show)
        
obj = Klass()
obj.show()
print(obj.show)
print(Klass.show)
print(id(obj.show))




print(obj.show_static)
print(id(Klass.show))
print(id(Klass.show))
print(id(Klass.show))
print(id(Klass.show))
print(id(Klass.show))
print(Klass.show_static)
print(id(obj.show_static))
print(id(Klass.show_static))

<class 'classmethod'>
<class 'staticmethod'>
klass
<bound method Klass.show of <class '__main__.Klass'>>
<bound method Klass.show of <class '__main__.Klass'>>
2838289806528
<function Klass.show_static at 0x00000294D784BAF0>
2838289806528
2838289784384
2838289806528
2838289784384
2838289806528
<function Klass.show_static at 0x00000294D784BAF0>
2838294215408
2838294215408


In [79]:
class_definition = """\
class myclass:

    def __init__(self):
        pass
        
    def show(self):
        print('abc')
"""

typename = "myclass"
namespace = dict(__name__='namedtuple_%s' % typename)
exec(class_definition, namespace)
result = namespace[typename]

print(result)
print(namespace)


<class 'namedtuple_myclass.myclass'>
{'__name__': 'namedtuple_myclass', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs>, 'all': <built-in function all>, 'any': <built-in function any>, 'ascii': <built-in function ascii>, 'bin': <built-in function bin>, 'breakpoint': <built-in function breakpoint>, 'callable': <built-in function callable>, 'chr': <built-in function chr>, 'compile': <built-in function compile>, 'delattr': <built-in function delattr>, 'dir': <built-in function dir>, 'divmod': <built-in function divmod>, 'eval': <built-in function eval>, 'ex

AttributeError: type object 'myclass' has no attribute '__code__'

In [81]:
def func():
    pass

print(func.__code__)

<code object func at 0x00000294D71E3BE0, file "<ipython-input-81-ea9d805fed58>", line 1>


Attempt to write a class decorator to set the storage name after the managed class is created. This removes the need to supply the name of the managed attribute when instantiate a descriptor instance

In [82]:
class descriptor:
    
    def __set__(self, instance, value):
        instance.__dict__[self.storage_name] = value
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.storage_name]
    
class myclass:
    attr = descriptor()
    
    def __init__(self, value):
        self.attr = value
        
obj = myclass('Initial value')
print(obj.attr)

AttributeError: 'descriptor' object has no attribute 'storage_name'

The error is well expected, the `descriptor instance` doesn't know where to store the value without `storage name`.

In [88]:
def NameDecorator(cls):
    # Inspect the class, look for managed attribute
    for attr_name, obj in cls.__dict__.items():
        if hasattr(obj, '__get__'):
            # Make sure this is our descriptor class which has 'storage_name' attr
            if hasattr(obj, 'storage_name'):
                # Tell the descriptor to store value under storage attribute
                # that has the same name with managed attribute
                # the attr_name we're looking at here is the managed attribute
                # don't be mistaken with the storage attribute which is an instance attribute
                obj.storage_name = attr_name
            
    return cls

class descriptor:
    
    def __init__(self):
        self.storage_name = None
    
    def __set__(self, instance, value):
        instance.__dict__[self.storage_name] = value
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.storage_name]
    
@NameDecorator
class myclass:
    attr = descriptor()
    
    def __init__(self, value):
        self.attr = value
        
obj = myclass('Initial value')
print(obj.attr)
print(obj.__dict__)

Initial value
{'attr': 'Initial value'}


Voila!!! By inspecting the class after it's created, we can find the managed attribute name and give it to descriptor instance.

This seems to be complex at first due to the relationships between the descriptor, the class, and the instances of both. Once we have a clear understanding of this gang, it's an sophisticated yet elegant solution.

Now, let's try another solution with a metaclass.

In [97]:
class MyMetaDeco(type):  # a Metaclass can be defined as easily as subclassing type
    
    # it's a convention to name the first argument of __init__ 
    # to be "cls" instead of "self" to make it clear that
    # the instance being modified is a class, not an instance of a class
    def __init__(cls, name, base, dic):  
        # Everything else should be the same with our decorator
        for attr_name, obj in cls.__dict__.items():
            if hasattr(obj, '__get__'):
                # Make sure this is our descriptor class which has 'storage_name' attr
                if hasattr(obj, 'storage_name'):
                    # Tell the descriptor to store value under storage attribute
                    # that has the same name with managed attribute
                    # the attr_name we're looking at here is the managed attribute
                    # don't be mistaken with the storage attribute which is an instance attribute
                    obj.storage_name = attr_name
                
class descriptor:
    
    def __init__(self):
        self.storage_name = None
    
    def __set__(self, instance, value):
        instance.__dict__[self.storage_name] = value
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.storage_name]
    
    
class myclass(metaclass=MyMetaDeco):  # define the metaclass to be our metaclass, default will be the class "type"
    attr = descriptor()
    
    def __init__(self, value):
        self.attr = value
        
obj = myclass('Initial value')
print(obj.attr)
print(obj.__dict__)

Initial value
{'attr': 'Initial value'}


Same effect has been achieved. The difference, you may ask?
Any subclass of `myclass` will also have this convenient feature.

In [102]:
class MyMetaDeco(type):  # a Metaclass can be defined as easily as subclassing type
    
    # it's a convention to name the first argument of __init__ 
    # to be "cls" instead of "self" to make it clear that
    # the instance being modified is a class, not an instance of a class
    def __init__(cls, name, base, dic):  
        # Everything else should be the same with our decorator
        for attr_name, obj in cls.__dict__.items():
            if hasattr(obj, '__get__'):
                # Make sure this is our descriptor class which has 'storage_name' attr
                if hasattr(obj, 'storage_name'):
                    # Tell the descriptor to store value under storage attribute
                    # that has the same name with managed attribute
                    # the attr_name we're looking at here is the managed attribute
                    # don't be mistaken with the storage attribute which is an instance attribute
                    obj.storage_name = attr_name
                
class descriptor:
    
    def __init__(self):
        self.storage_name = None
    
    def __set__(self, instance, value):
        instance.__dict__[self.storage_name] = value
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.storage_name]
    
    
class myclass(metaclass=MyMetaDeco):  # define the metaclass to be our metaclass, default will be the class "type"
    attr = descriptor()
    
    def __init__(self, value):
        self.attr = value
        
        
# Create a subclass of myclass,
# the subclass will also be able
# to register the name of managed attribute
# with the descriptor instance
class mysub(myclass):
    subattr = descriptor()
    
    def __init__(self, value):
        self.subattr = value
        
obj = mysub('Initial value')
print(obj.subattr)
print(obj.__dict__)

Initial value
{'subattr': 'Initial value'}


 This cannot be achieved with decorator, since the decorator only affect the decorated class.

In [100]:
def NameDecorator(cls):
    # Inspect the class, look for managed attribute
    for attr_name, obj in cls.__dict__.items():
        if hasattr(obj, '__get__'):
            # Make sure this is our descriptor class which has 'storage_name' attr
            if hasattr(obj, 'storage_name'):
                # Tell the descriptor to store value under storage attribute
                # that has the same name with managed attribute
                # the attr_name we're looking at here is the managed attribute
                # don't be mistaken with the storage attribute which is an instance attribute
                obj.storage_name = attr_name
            
    return cls

class descriptor:
    
    def __init__(self):
        self.storage_name = None
    
    def __set__(self, instance, value):
        instance.__dict__[self.storage_name] = value
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.storage_name]
    
@NameDecorator
class myclass:
    attr = descriptor()
    
    def __init__(self, value):
        self.attr = value
        
class mysub(myclass):
    subattr = descriptor()
    
    def __init__(self, value):
        self.subattr = value
        
myclass_obj = myclass('Initial value')
print(myclass_obj.attr)
print(myclass_obj.__dict__)

obj = mysub('Initial value')
print(obj.subattr)
print(obj.__dict__)

Initial value
{'attr': 'Initial value'}
Initial value
{None: 'Initial value'}


A couple of things going on here:
1. when `obj` is instantiated, `__init__` is invoked to assign 'Initial value' to `subattr`
2. since we have a managed attribute with same name, i.e. `subattr`, the descriptor's `__set__` is invoked to handle the assignment
3. However, the assignment was done by assigning the string 'Initial value' to `obj.__dict__` under the name `None` because by default, the descriptor instance's `storage_name` is `None`
4. When we're retrieving `obj.subattr`, the descriptor `__get__` was invoked to retrieve the value from `obj.__dict__`, looking for the `storage_name` `None` which does exist. Hence, the value `Initial value` was retrieved.
5. Inspecting the `obj.__dict__` clearly shows that the value was not stored under the same name with `store attribute`. This is different from how the `attr` of `myclass` was stored.

In [116]:
class MyMeta(type):
    print("<1> MyMeta body")
    
    @classmethod
    def __prepare__(cls, name, bases):
        print("<10> __prepare__")
        return dict()
    
    def __new__(self, cls, bases, dic):
        print(f"<2> __new__({self}, {cls!r}, {bases}, {dic})")
        return super().__new__(self, cls, bases, dic)
    
    def __init__(self, cls, bases, dic):
        print(f"<3> __init__({self}, {cls}, {bases}, {dic})")
        
        super().__init__(cls, bases, dic)
        
class MyClass(metaclass=MyMeta):
    print("<4> MyClass body")
    print("Something else")
    
    def myfunc(self):
        pass
    
    print(myfunc)
    print(MyClass)
    
print(MyClass)

<1> MyMeta body
<10> __prepare__
<4> MyClass body
Something else
<function MyClass.myfunc at 0x00000294D7D93790>
<class '__main__.MyClass'>
<2> __new__(<class '__main__.MyMeta'>, 'MyClass', (), {'__module__': '__main__', '__qualname__': 'MyClass', 'myfunc': <function MyClass.myfunc at 0x00000294D7D93790>})
<3> __init__(<class '__main__.MyClass'>, MyClass, (), {'__module__': '__main__', '__qualname__': 'MyClass', 'myfunc': <function MyClass.myfunc at 0x00000294D7D93790>})
<class '__main__.MyClass'>


Observe the order in which a class is defined above.